In [1]:
from docx import *
from docx.enum.text import WD_COLOR_INDEX
import pandas as pd
import re
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

factory = StemmerFactory()
stemmer = factory.create_stemmer()

In [2]:
# read file
# document = Document("Raw/V-Z-Prep.docx")
# document = Document("Data/2. Preprocessed/W-Prep.docx")
document = Document("execution.docx")

In [3]:
def cleanWord(word) :
    #cek spasi pada kata (jika terdiri dari 2 kata)
    if re.search("\s", word) :
        word = word.split()

        word[:] = [w for w in word if not (w in roman_numerals or re.match("\s\d\s|\s\d|\d\s|\d", w))]
    
        word = " ".join(word)
        
        if len(word) < 1 :
            word = None

    #jika terdiri dari 1 kata
    else :
     
        if word in roman_numerals or re.search("\d", word) :
            word = None
    
    return(word)

def replaceData (x) :
    for idx in range(len(x)):
        x[idx] = re.sub("¤","fi",x[idx])
        x[idx] = re.sub("¶","fl",x[idx])
        x[idx] = re.sub("\s-\s","-",x[idx])
        x[idx] = re.sub("-\s","",x[idx])
        x[idx] = re.sub("\s{2}|\s{3}"," ",x[idx])
        x[idx] = re.sub("\s*$|\xad|\xad\s*","",x[idx])
        x[idx] = re.sub("^\)|^\)\s*","",x[idx])
        x[idx] = re.sub("\(\s\)|\(\s=\)|\(=\)|\(=$|\($|\=$","",x[idx])
        x[idx] = re.sub("^\w\)\s*|\s\w\)$","",x[idx])
        x[idx] = re.sub("^\s*|^\s|^\s\w","",x[idx])
        x[idx] = re.sub("\[and","",x[idx])
        x[idx] = re.sub("\[|\[\s","",x[idx])
        x[idx] = re.sub("\]","",x[idx])
        x[idx] = re.sub("\s-\s|-\s","-",x[idx])
        x[idx] = re.sub("\s<\s|\s<|<\s|<","",x[idx])
        x[idx] = re.sub("^\s","",x[idx])
        x[idx] = re.sub("\s\)",")",x[idx])
        x[idx] = re.sub("\(\s","(",x[idx])
        x[idx] = re.sub("^/.*/","",x[idx])
        x[idx] = re.sub("\s/\s|\s/|/\s","/",x[idx])
        x[idx] = re.sub("\snya","nya",x[idx])
        x[idx] = re.sub("\skan$","kan",x[idx])
        x[idx] = re.sub("\skan\s","kan ",x[idx])

        x[idx] = re.sub("\.$|\s\.$|\.\s$","",x[idx])
        x[idx] = re.sub("\.\.$","...",x[idx])
        x[idx] = re.sub("\s\.\s",". ",x[idx])
        x[idx] = re.sub("\s\,\s",", ",x[idx])
        
        x[idx] = re.sub("someone ’s","someone’s",x[idx])
        x[idx] = re.sub("ke pada","kepada",x[idx])
        x[idx] = re.sub("^se ","se",x[idx])
        
        x[idx] = re.sub("_","...",x[idx])
    
    return x

# fungsi untuk menyimpan hasil ke bentuk list
def cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan):
    x = [lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan]
    
    x = replaceData(x)
    
    #split panah
    if x[4] != "" :        
        x[4] = re.split(",|;",x[4])
        listPanah = []
        
        for idx in range(len(x[4])):
            if '(' in x[4][idx] :
                x[4][idx], y = re.sub(r"[/(].*?[/))=]","",x[4][idx]),re.sub(r"[()=]","",x[4][idx])
                x[4][idx], y = re.sub(r"\s$","",x[4][idx]),re.sub(r"\s$","",y)
                x[4][idx] = re.sub("\s{2}|\s{3}"," ",x[4][idx])
#                 x[4].append(x)
                listPanah.append(y)
        
        if len(listPanah) != 0 :
            for val in listPanah :
                x[4].append(val)
    
    if x[7] != "" :
        x[7] = re.split(",|;",x[7])
        
    
    #split terjemahan
    if x[9] != "" :
        x[9] = re.split(",|;",x[9])
        x[9] = replaceData(x[9])
        
        if "" in x[9] :
            x[9].remove("")
        
        for idx in range(len(x[9])):
#             x[9][idx] = re.sub("_","...",x[9][idx])
            x[9][idx] = re.sub("e\*g\*","e.g.,",x[9][idx])
            x[9][idx] = re.sub("i\*e\*","i.e.,",x[9][idx])
            x[9][idx] = re.sub("\*etc",", etc",x[9][idx])
            x[9][idx] = re.sub("etc \.","etc.",x[9][idx])
            x[9][idx] = re.sub("\*|\s\*|\*\s|\s\*\s",",",x[9][idx])
            
        if len(x[9]) < 1 :
            x[9] = ""
        elif len(x[9]) == 1 and x[9][0] == "" :
            x[9] = ""
    
    y = pd.DataFrame(data=[x],columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
    
    return x,y

def cleanParenthesis(word) :
    if '(' in word :
        
#         if (re.search("\(=", word) ) :
#             x, y = re.sub(r"[\(=].*?[\)]","",word),re.sub(r".*\(=|\).*","",word)
#         else :
#             x, y = re.sub(r"[\(].*?[\)]","",word),re.sub(r"[()]","",word)

        if (re.search("\(=", word) ) :
            x, y = re.sub(r"[\(=].*?[\)]","",word),re.sub(r"[(=)]","",word)
        else :
            x, y = re.sub(r"[\(].*?[\)]","",word),re.sub(r"[()]","",word)

        
        x, y = re.sub(r"\s$","",x),re.sub(r"\s$","",y)
        word = x
        similar = y
        tempLemaSublema = word
        boolLemaSublema = True
    else :
        word = re.sub(r"[()]|\s$","",word)
        similar = ""
        tempLemaSublema = word
        boolLemaSublema = True
        
    return word,similar,tempLemaSublema,boolLemaSublema

def readItalicData(word,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan) :     
    if word == "opp" or word == "cp":
        word = ""

    #KETERANGAN
    # mengambil keterangan
    if index > 0 and re.search("\(", paragraph.runs[index-1].text) and re.search("–|~", word) == None:

        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)
            
        else :
            
            if (index+1 < len(paragraph.runs) and paragraph.runs[index+1].italic) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) :
                tempGabungan.append(word)
            else :
                if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = word
                elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                    terjemahan = terjemahan+" "+word

        # update sementara
        try :
            if (re.search("\)", word) and re.search("–|~", word) == None) or (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text) and re.search("–|~", word) == None):
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
        except :
            if re.search("\)", word) and re.search("–|~", word) == None:
                keterangan = " ".join(tempKeterangan)
                tempKeterangan = []
                
    # mengambil keterangan selanjutnya jika terdiri dari > 1 keterangan
    elif len(tempKeterangan) > 0 and word not in tempKeterangan:
        if re.search("\s", word) :
            split_text = word.split()
            tempWord = []

            for s in split_text :
                if s in list_ket :
                    tempWord.append(s)
                else :
                    break

            word = " ".join(tempWord)

            tempKeterangan.append(word)

        elif word in list_ket :
            tempKeterangan.append(word)

            try :
                if (index+1 < len(paragraph.runs) and re.search("\)", paragraph.runs[index+1].text)) or re.search("\)", word) :
                    keterangan = " ".join(tempKeterangan)
                    tempKeterangan = []
            except :
                print("terjadi kesalahan")
                
        else :
            if terjemahan == "" and word not in roman_numerals and boolPanah != True:
                terjemahan = word
            elif terjemahan != "" and word not in roman_numerals and boolPanah != True:
                terjemahan = terjemahan+" "+word
                
    else :
        tempGabungan.append(word)

    return keterangan,tempKeterangan,tempGabungan,terjemahan

In [26]:
lexicon = pd.DataFrame(columns=["lema","sublema","gabungan","similar","panah","akronim","antonim","perbandingan","keterangan","terjemahan"])
# Hapus dari list jika huruf yang dieksekusi == huruf romawi
roman_numerals = ["I","II","III","IV","V","VI","VII","VIII","IX","X"]
list_ket = ["A","abbr","Ac","acr","adj","AE","anat","app","Bal","Ban","Bat","BD","BE","BG","bio","biol","bot","C","chem","cla","col","coq","cp","D","D/E","derog","E","elec","e.o.","epist","esp","euph","exclam","fin","Fr","G","geo","geol","Gr","gram","Hind","IBT","infr","init","insur","Irja","Isl","J","J/Jv","joc","Jp","Jv","K","k.o.","L","leg","ling","lit","M","Mad","Mal","math","Med","med","mil","Min","mod","mus","naut","NTB","NTT","ob","obj","O jv","onom","opp","o.s.","Pal","Pap","Pers","petro","phys","pl","pl obj","pl subj","poet","Port","Pr","pron","q.v.","RC","reg","rel","rev","S","sg","Sg","Skr","Skr neo","sl","s.o.","s.o.'s","s.t.","stat","subj","Sum","Tag","Tam","usu","voc","vulg","zod","neo"]
list_data = ["something", "someone", "oneself", "each other","something ", "someone ", "oneself ", "each other "]
list_and = ["and", " and", "and ", " and ", "[and", "[and "]

#variabel
lema = ""
sublema = ""
gabungan = ""
similar = ""
panah = ""
akronim = ""
antonim = ""
perbandingan = ""
keterangan = ""
terjemahan = ""

boolLemaSublema = False
boolPanah = False
boolSimilar = False
boolAntonim = False
tempLema = ""
tempSublema = ""
tempTerjemahan = ""
tempGabungan = []
tempKeterangan = []
tempAkronim = []
index = 0

#looping paragraf tiap document
for paragraph in document.paragraphs:
    
    boolLemaSublema = False
    
    index = 0
    
    #menyimpan nilai indentasi (first indent dan left indent)
    first = paragraph.paragraph_format.first_line_indent
    left = paragraph.paragraph_format.left_indent
    
    #looping setiap kata di setiap paragrafnya
    for run in paragraph.runs:
        
#         print("prev = "+paragraph.runs[index-1].text)
#         print("word = "+run.text)
#         print(index)
#         print(run.font.all_caps)
        
  
        #mengatasi left indent == none, dengan mengubah nilai variabel left dengan tempLeft (nilai pada left sebelumnya)
        if left == None :
            left = 228600
        
        #deteksi kata bold
        if run.bold and run.italic == None  :
            if run.text.isspace() == False :
                #baca indent lema
                
#                 print("word = "+run.text)
#                 print(index)
#                 print(paragraph.runs[index-1].text)

                if (((first == None or first == 0 or first == 76200 or first == -635) and (left <= 76835)) or ((first == -152400 or first == -153035) and (left >= 227965 or left <= 76835))) :      

#                     print("lema = "+run.text)
            
#                     try :
#                         print("prev-2 = "+paragraph.runs[index-2].text)
#                     except :
#                         print("prev-2 = -")
                        
#                     print("prev = "+paragraph.runs[index-1].text)
#                     print("word = "+run.text)

                    tempLeft = left

                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                        
                        data = False
                        for item in x[0]:
                            if item != "" :
                                data = True
                                
                        if data == True :
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                    
                    
                    #variabel 
                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""

                    boolPanah = False
                    boolSimilar = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []

                    word = cleanWord(run.text)

                    if word != None :
                        
                        #LEMA
                        #jika lema kosong atau belum terdapat lema, lema = word
#                         if boolLemaSublema == False and (index > 0 and not re.search("\=", paragraph.runs[index-1].text)) and (index > 1 and not re.search("\=", paragraph.runs[index-2].text)) :
                        if boolLemaSublema == False :
                            
#                             print (boolLemaSublema)
#                             print(index)
#                             print(word)
#                             print(paragraph.runs[index-1].text)
#                             print ((index > 0 and not re.search("\=", paragraph.runs[index-1].text)))
#                             print(paragraph.runs[index-2].text)
#                             print ((index > 1 and not re.search("\=", paragraph.runs[index-2].text)))
#                             print("ini lho ="+run.text)
                            
                            # jika similar
                            if (index > 0 and not re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and not re.search("\=", paragraph.runs[index-2].text)):
                                similar = word
                    
#                                 print(word)
                    
                                if '(' in similar :
                                    listTemp = []
                                    results = cleanParenthesis(similar)

                                    listTemp.append(results[0])
                                    listTemp.append(results[1])
#                                     listTemp[0] = results[0]
#                                     listTemp[1] = results[1]

                                    for temp in listTemp :
                                        similar = temp

                                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                            data = False
                                            for item in x[0]:
                                                if item != "" :
                                                    data = True

                                            if data == True :
                                                print(x[0])
                                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                                        lema = ""
                                        gabungan = ""
                                        similar = ""
                                        panah = ""
                                        akronim = ""
                                        antonim = ""
                                        perbandingan = ""
                                        keterangan = ""
                                        terjemahan = ""
                                        boolSimilar = False
                                        boolAntonim = False
        
                            # jika lema
                            else :

                                results = run.text
                                words = results.split()

                                if (len(words) > 2 and (words[1] in roman_numerals or re.match("\d", words[1]))) :
                                    lema = words[0]
                                    sublema = words[2]

                                    results = cleanParenthesis(lema)
                                else :
                                    results = cleanParenthesis(word)

                                lema = results[0]
                                similar = results[1]
                                boolLemaSublema = [3]

                                tempLema = lema
                                tempSublema = sublema

                        #jika lema tidak kosong atau variabel lema sudah terisi
                        else :
                            
                            #SIMILAR (AND)
                            #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                            if (index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\=", paragraph.runs[index-2].text)) :
                                
                                similar = word
                                    
                                if '(' in similar :
                                    listTemp = []
                                    results = cleanParenthesis(similar)

                                    listTemp.append(results[0])
                                    listTemp.append(results[1])
#                                     listTemp[0] = results[0]
#                                     listTemp[1] = results[1]

                                    for temp in listTemp :
                                        similar = temp

                                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                            data = False
                                            for item in x[0]:
                                                if item != "" :
                                                    data = True

                                            if data == True :
                                                print(x[0])
                                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                                        lema = ""
                                        gabungan = ""
                                        similar = ""
                                        panah = ""
                                        akronim = ""
                                        antonim = ""
                                        perbandingan = ""
                                        keterangan = ""
                                        terjemahan = ""
                                        boolSimilar = False
                                        boolAntonim = False
                                
                            #jika bukan similar
                            else :  
                                #SUBLEMA(JIKA SEBARIS/SEPARAGRAF DENGAN LEMA)  
                                #jika sublema kosong
                                if sublema == "" :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                       
                                        data = False
                                        for item in x[0]:
                                            if item != "" :
                                                data = True

                                        if data == True :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)
                                    
                                    lema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False
                                    boolAntonim = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                                #jika sublema tidak kosong, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                    
                                        data = False
                                        for item in x[0]:
                                            if item != "" :
                                                data = True

                                        if data == True :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)
                                    
                                    lema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False
                                    boolAntonim = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                #baca indent sublema
                else :
                    
#                     print("sublema = "+run.text)
                    
                    tempLeft = left 

                    #deteksi kata bold
                    if(run.bold) and run.text.isspace() == False :
    
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                            
                            data = False
                            for item in x[0]:
                                if item != "" :
                                    data = True
                                
                            if data == True :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)
                            
                        #variabel
                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""

                        boolPanah = False
                        boolSimilar = False
                        boolAntonim = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                        word = cleanWord(run.text)

                        if word != None :

                            #SUBLEMA
                            #jika sublema kosong
                            if boolLemaSublema == False :

                                if (index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\=", paragraph.runs[index-2].text)):
                                    similar = word
                                    
                                    if '(' in similar :
                                        listTemp = []
                                        results = cleanParenthesis(similar)

                                        listTemp.append(results[0])
                                        listTemp.append(results[1])
                                        
                                        for temp in listTemp :
                                            similar = temp
                                            
                                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                                data = False
                                                for item in x[0]:
                                                    if item != "" :
                                                        data = True

                                                if data == True :
                                                    print(x[0])
                                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                                            lema = ""
                                            similar = ""
                                            gabungan = ""
                                            similar = ""
                                            panah = ""
                                            akronim = ""
                                            antonim = ""
                                            perbandingan = ""
                                            keterangan = ""
                                            terjemahan = ""
                                            boolSimilar = False
                                            boolAntonim = False
                                else :
                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

                            #jika sublema tidak kosong
                            else :
                                
#                                 print("disini = "+word)
                                
                                #SIMILAR (AND)
                                #jika kata sebelum kata style bold adalah "and" maka dianggap sebagai similar
                                if (index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and) or (index > 0 and re.search(",", paragraph.runs[index-1].text)) or (index > 1 and re.search(",", paragraph.runs[index-2].text)) or (index > 0 and re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\=", paragraph.runs[index-2].text)):
                                    similar = word
                                    
#                                     print(word)
                                    
                                    if '(' in similar :
                                        listTemp = []
                                        results = cleanParenthesis(similar)

                                        listTemp.append(results[0])
                                        listTemp.append(results[1])
                                        
                                        for temp in listTemp :
                                            similar = temp
                                            
                                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                                                data = False
                                                for item in x[0]:
                                                    if item != "" :
                                                        data = True

                                                if data == True :
                                                    print(x[0])
                                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                                            lema = ""
                                            similar = ""
                                            gabungan = ""
                                            similar = ""
                                            panah = ""
                                            akronim = ""
                                            antonim = ""
                                            perbandingan = ""
                                            keterangan = ""
                                            terjemahan = ""
                                            boolSimilar = False
                                            boolAntonim = False

                                #SUBLEMA (JIKA SUBLEMA SEBARIS/SEPARAGRAF DENGAN SUBLEMA YANG LAIN)
                                #jika bukan, maka simpan sublema ke list terlebih dahulu
                                else :
                                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                        
                                        data = False
                                        for item in x[0]:
                                            if item != "" :
                                                data = True

                                        if data == True :
                                            print(x[0])
                                            lexicon  = lexicon.append([x[1]],ignore_index=True)
                                    
                                    lema = ""
                                    sublema = ""
                                    gabungan = ""
                                    similar = ""
                                    panah = ""
                                    akronim = ""
                                    antonim = ""
                                    perbandingan = ""
                                    keterangan = ""
                                    terjemahan = ""
                                    boolSimilar = False
                                    boolAntonim = False

                                    results = cleanParenthesis(word)

                                    sublema = results[0]
                                    similar = results[1]
                                    tempSublema = results[2]
                                    boolLemaSublema = [3]

        #deteksi kata italic (kata gabungan, keterangan)
        elif (run.italic and run.bold) or run.italic :
            if run.text.isspace() == False :
                
                if (index > 0 and paragraph.runs[index-1].text in list_and) or (index > 1 and paragraph.runs[index-2].text in list_and) or (index > 0 and re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\=", paragraph.runs[index-2].text)) :
                    boolSimilar = True
                
#                 print(boolSimilar)
#                 print(run.text)
                
                if boolSimilar == True and similar != "" :
                    similar = similar + " " + run.text

                if (index > 0 and (list(filter(lambda x : x == True, list(map(lambda x : paragraph.runs[index-1].text.endswith(x), list_data)))))) or (index > 1 and (list(filter(lambda x : x == True, list(map(lambda x : paragraph.runs[index-1].text.endswith(x), list_data)))))) or ((index > 0 and re.search("_|;", paragraph.runs[index-1].text)) or (index > 1 and re.search("_|;", paragraph.runs[index-2].text))) :
 
                    if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                        x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                        data = False
                        for item in x[0]:
                            if item != "" :
                                data = True
                                
                        if data == True :
                            print(x[0])
                            lexicon  = lexicon.append([x[1]],ignore_index=True)

                    lema = ""
                    sublema = ""
                    gabungan = ""
                    similar = ""
                    panah = ""
                    akronim = ""
                    antonim = ""
                    perbandingan = ""
                    keterangan = ""
                    terjemahan = ""
                            
                    boolPanah = False
                    boolSimilar = False
                    boolAntonim = False
                    tempGabungan = []
                    tempKeterangan = []
                    tempAkronim = []
                    
                    results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

                    keterangan = results[0]
                    tempKeterangan = results[1]
                    tempGabungan = results[2] 
                    terjemahan = results[3]
                    
#                     print("if ="+run.text)
                
                elif boolAntonim == True :
                    antonim = antonim+" "+run.text
#                     print("elif1 ="+run.text)
                    
                elif boolPanah == False and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :
                    word = re.sub("\.","", run.text)
                    word = cleanWord(word)

                    if word != None :
                        panah = word
                        boolPanah = True
                        word = ""
                    
#                     print("elif2 ="+run.text)

                else :
                    results = readItalicData(run.text,index,paragraph,keterangan,tempKeterangan,tempGabungan,terjemahan)

                    keterangan = results[0]
                    tempKeterangan = results[1]
                    tempGabungan = results[2] 
                    terjemahan = results[3]
                    
#                     print("else ="+run.text)
                    
#                 try :
#                     print(results)
#                 except : 
#                     print("-")

#                 print(tempGabungan)

        # deteksi kata reguler (akronim, terjemahan)
        else : 
            
#             print(tempGabungan)

            if run.text.isspace() == False :
                word = run.text

                if word != None :
                    #GABUNGAN
                    
                    temp = " ".join(tempGabungan)
#                     print(temp)

                    if re.search("–", temp) :
                        gabungan = re.sub("–",tempLema,temp)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
#                         print(temp)
#                         gabungan = results[0]
#                         similar = results[1]
                            
                        if (boolSimilar != True) or re.search("\(", gabungan) :
                            gabungan = results[0]
                            similar = results[1]
                        else :
                            gabungan = ""
                            similar = results[0]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            data = False
                            for item in x[0]:
                                if item != "" :
                                    data = True
                                
                            if data == True :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        boolSimilar = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                    elif re.search("~", temp) :
                        gabungan = re.sub("~",tempSublema,temp)
                        gabungan = re.sub("  "," ", gabungan)

                        results = cleanParenthesis(gabungan)
                        
#                         gabungan = results[0]
#                         similar = results[1]
                            
                        if (boolSimilar != True) or re.search("\(", gabungan) :
                            gabungan = results[0]
                            similar = results[1]
                        else :
                            gabungan = ""
                            similar = results[0]
                        
                        if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                            x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)

                            data = False
                            for item in x[0]:
                                if item != "" :
                                    data = True
                                
                            if data == True :
                                print(x[0])
                                lexicon  = lexicon.append([x[1]],ignore_index=True)

                        lema = ""
                        sublema = ""
                        gabungan = ""
                        similar = ""
                        panah = ""
                        akronim = ""
                        antonim = ""
                        perbandingan = ""
                        keterangan = ""
                        terjemahan = ""
                            
                        boolPanah = False
                        boolSimilar = False
                        boolAntonim = False
                        tempGabungan = []
                        tempKeterangan = []
                        tempAkronim = []

                    else :
                        
#                         print(tempGabungan)
                        if len(tempGabungan) != 0:
                            
                            if boolPanah == True and panah != word and word not in roman_numerals :
                                panah = panah+" "+temp

                                temp = ""
                                tempGabungan = []
                                
                            elif boolSimilar == True and similar != word and word not in roman_numerals :
                                similar = similar+" "+temp

                                temp = ""
                                tempGabungan = []
                            
#                             elif len(tempGabungan) > 1 and ((tempLema != "" and tempLema.lower() in temp.lower()) or ( tempSublema != "" and tempSublema.lower() in temp.lower())) :
                            elif len(tempGabungan) > 0 and ((tempLema != "" and tempLema.lower() in temp.lower()) or ( tempSublema != "" and tempSublema.lower() in temp.lower())) :
    
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

#                                 gabungan = results[0]
#                                 similar = results[1]
                        
                                if (boolSimilar != True) or re.search("\(", gabungan) :
                                    gabungan = results[0]
                                    similar = results[1]
                                else :
                                    gabungan = ""
                                    similar = results[0]

                                boolSimilar = False
                                tempGabungan = []
                            
#                           // jika nama latin
                            elif (index > 1 and len(tempGabungan) <= 3) or (index == len(paragraph.runs)-1 and  len(tempGabungan) > 1) :
                                if (boolSimilar != True) :
                                    terjemahan = terjemahan+" "+temp
                                else :
                                    similar = temp
                                
                                boolSimilar = False
                                temp = ""
                                tempGabungan = []
            
                            else :
                            
                                gabungan = gabungan+" "+temp
                                gabungan = re.sub(r'  ',' ', gabungan)

                                results = cleanParenthesis(gabungan)

#                                 gabungan = results[0]
#                                 similar = results[1]
                        
                                if (boolSimilar != True) or re.search("\(", gabungan) :
                                    gabungan = results[0]
                                    similar = results[1]
                                else :
                                    gabungan = ""
                                    similar = results[0]

                                boolSimilar = False
                                tempGabungan = []

                
                #AKRONIM
                # mengambil akronim
                if (index > 0 and re.search("\[", paragraph.runs[index-1].text)):
                    tempAkronim.append(word)

                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", paragraph.runs[index].text) :
                        akronim = " ".join(tempAkronim)
                        tempAkronim = []

                # mengambil akronim selanjutnya jika terdiri > 1 kata
                elif len(tempAkronim) > 0 and word not in tempAkronim :
                    tempAkronim.append(word)

                    if (index+1 < len(paragraph.runs) and re.search("\]", paragraph.runs[index+1].text)) or re.search("\]", paragraph.runs[index].text) :
                        akronim = " ".join(tempAkronim)
                        akronim = re.sub(r'  ',' ', akronim)
                        tempAkronim = []
                        
                else :
                    
                    #PANAH   
                    if boolPanah == True and word not in roman_numerals :
                        tempWord = re.sub("\.","", word)
                        tempWord = cleanWord(tempWord)
                     
                        if tempWord != None :
                            panah = panah+" "+tempWord
                            word = ""
                    
                    if boolPanah == True and re.search("\.", word) :

                        panah = re.sub("\.|\s\.","",panah)
                        boolPanah = False
                        word = ""

                    if word != "" and word != "." and ((index > 0 and re.search(">", paragraph.runs[index-1].text)) or (index > 1 and re.search(">", paragraph.runs[index-2].text))) :

                        word = re.sub("\.","", word)
                        word = cleanWord(word)

                        if word != None :
                            panah = word
                            boolPanah = True
                            word = ""
                            
#                     elif word != "" and word != "." and ((index > 0 and re.search("\=", paragraph.runs[index-1].text)) or (index > 1 and re.search("\=", paragraph.runs[index-2].text))) :

                        
#                         word = re.sub("\.","", word)
#                         word = cleanWord(word)

#                         if word != None :
                            
#                             if similar != "" :
#                                 similar = similar + " " + word
#                             else :
#                                 similar = word
                            
#                             boolSimilar = True
#                             word = ""

                    elif boolAntonim == False and (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "opp") or (index > 1 and paragraph.runs[index-2].text == "opp")):
                        re.sub("\s","",word)

                        antonim = word
                        boolAntonim = True
                
                    elif (index > 0 and paragraph.runs[index-1].italic == True) and ((index > 0 and paragraph.runs[index-1].text == "cp") or (index > 1 and paragraph.runs[index-2].text == "cp")):
                        re.sub("\s","",word)

                        perbandingan = word

                    else :

                        #TERJEMAHAN
                        if re.search("\]\s", word) :
                            word = re.sub("\]\s","",word)

                        if re.search("and", word) and ((index > 0 and (paragraph.runs[index-1].bold) == True or paragraph.runs[index-1].italic or re.search("\]", paragraph.runs[index-1].text)) or (index > 1 and re.search("\]", paragraph.runs[index-2].text))) and ((index+1 < len(paragraph.runs) and (paragraph.runs[index+1].bold == True or paragraph.runs[index+1].italic == True)) or (index+2 < len(paragraph.runs) and (paragraph.runs[index+2].bold == True or paragraph.runs[index+2].italic == True))):
                            word = ""

                        if word == "[" or word == "]" or re.search(">", word) :
                            word = ""

                        if terjemahan == "" and boolPanah != True :
                            terjemahan = word
                            
                        elif terjemahan != "" and boolPanah != True:
                            terjemahan = terjemahan+" "+word

#                         print(run.text)
                        if  (((re.search("\.$|\.\”$", run.text)) and index == len(paragraph.runs)-1) or (re.search("\!|\?", run.text)) or re.match("\w\)", run.text)) and run.font.highlight_color != WD_COLOR_INDEX.BRIGHT_GREEN :
#                         if  ((re.search("\.$|\!|\?", run.text) or re.match("\w\)", run.text)) and index == len(paragraph.runs)-1) and run.font.highlight_color != WD_COLOR_INDEX.YELLOW :
        
                            if lema != "" or sublema != "" or gabungan != "" or similar != "" or panah != "" or akronim != "" or antonim!= "" or keterangan != "" or terjemahan != "":
                                x = cleanData(lema,sublema,gabungan,similar,panah,akronim,antonim,perbandingan,keterangan,terjemahan)
                                
                                data = False
                                for item in x[0]:
                                    if item != "" :
                                        data = True
                                
                                if data == True :
                                    print(x[0])
                                    lexicon  = lexicon.append([x[1]],ignore_index=True)

                            lema = ""
                            sublema = ""
                            gabungan = ""
                            similar = ""
                            panah = ""
                            akronim = ""
                            antonim = ""
                            perbandingan = ""
                            keterangan = ""
                            terjemahan = ""
                            
                            boolPanah = False
                            boolSimilar = False
                            boolAntonim = False
                            tempGabungan = []
                            tempKeterangan = []
                            tempAkronim = []
    
        index += 1                 

['tujuh', '', '', '', '', '', '', '', '', ['seven']]
['', '', 'tujuh belas', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['seventeen']]
['', '', 'tujuhbelasan', '', '', '', '', '', '', ['on August 17th', 'In donesian independence day']]
['', '', 'pidato upacara', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['speech given on that day']]
['', '', 'tujuh Agustus', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'ob', ['name of the Army division stationed in West Sumatra']]
['', '', 'tujuh bulan/hari', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['seven months/days']]
['', '', 'menujuh bulan/har i', '', '', '', '', '', '', ['an d menujuh-bulan i t o c e l e b r a t e t h e s e v e n t h month/day of pregnancy/death']]
['', '', 'tujuh keliling', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['the seven']]
['', '', '', '', '', '', '', '', '', ['rounds of the Kabah (part of the hajj ceremonies)']]
[

['', '', 'maju ujian', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['to take/stand for an exam']]
['', '', 'ujian akhir', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['posttest']]
['', '', 'ujian awal', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['pretest']]
['', '', 'ujian bakat', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['aptitude test']]
['', '', 'ujian dinas', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['trial period (for civil servants)']]
['', '', 'ujian hér', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['re-examination']]
['', '', 'ujian kandidat', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['B.A']]
['', '', '', '', '', '', '', '', '', ['exam (in the older system)']]
['', '', 'ujian kedinasan', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['service test']]
['', '', 'ujian kenaikan kelas', '', '', '', '', '', 

['', '', 'asal di darat ikan di laut bertemu dalam belanga juga', 'asal di darat ikan di laut garam di laut, asal di gunung bertemu dalam belanga juga', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['they are predestined for each other', 'their marriage was made in heaven']]
['baba', '', '', '', ['BABAH'], '', '', '', '', '']
['', '', 'baba Ali', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['sarcastic reference to Chinese businessman (= Baba) who runs a business using a license sold to him by an Indonesian (= Ali) to whom the license was issued']]
['', '', 'baba', '', '', '', '', ['ALI'], '', '']
['badé', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'Bal', ['a structure in the form of a mountain (= méru) with 11 levels and about 15 meters high (used for cremations)']]
['', 'bahara', '', '', '', '', '', '', 'Skr', '']
['', '', '', '', '', '', '', '', '', ['a weight which varies according to the item weighed']]
['', '', 'agar-a

['bi', '', '', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', 'A', ['in', 'at', 'on (place and time)', 'with (indicating connection/association/attention)', 'through', 'by means of (designating instrumentality or agency)', 'for (= at the price of)', 'by (= in the amount of)', 'by (introducing an oath)']]
['', '', 'bi hi', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['on/with him']]
['bilang', '', '', '', '', '', '', '', '', '']
['', '', '', '', ['BILANGAN'], '', '', '', 'coq', ['number (1', '2', '3, etc.)', 'figure', 'enumeration']]
['', '', 'dengan modal yang tidak lagi bilang juta, tetapi sudah milyaran', '', '', '', '', '', '', '']
['', '', '', '', '', '', '', '', '', ['with a capital no longer running into the millions', 'but already in the billions']]
['', '', '', '', '', '', '', '', '', ['= SEBILANG']]
['bowong', '', '', '', '', '', '', '', '', ['a person (= wong in Javanese) who pulls a plow to replace a water buffalo (= kebo in Javanese)']]
['

In [18]:
x = "scsd]”"
re.search("]", x)

<_sre.SRE_Match object; span=(4, 5), match=']'>

In [6]:
# lexicon.to_excel (r'Data/3. Scrapped/W-Results.xlsx', index = False, header=True)